<a href="https://colab.research.google.com/github/sabino975/projeto_imersao_alura_05/blob/main/projeto_imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instala a biblioteca Generative Ai
!pip install -q -U google-generativeai

In [ ]:
# Importação das bibliotecas utilizadas no projeto
import numpy as np
import pandas as pd
import json
import google.generativeai as genai
from google.colab import userdata

In [ ]:
# Configuração da chave utilizada pela biblioteca Generative Ai
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Configuração dos parâmetros utilizados pela biblioteca Generative Ai
#Parâmetros de geração
generation_config = {
  "candidate_count": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
  "temperature": 0.5,
  "response_mime_type": "application/json"
}
#Parâmetros de segurança
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]
# Instruções iniciais para o modelo montar o resultado da pesquisa em um formato definido
system_instruction = "Você é um assistente de pesquisa.  \nDeve pesquisar por temas relacionados as palavras chave informadas.\nO resultado da pesquisa deve ser exibido no seguinte formato:\n\nTítulo: Inteligência Artificial (IA) e Machine Learning \nDescrição: Como a IA está impulsionando a tomada de decisão em carros autônomos, incluindo navegação, detecção de objetos e aprendizado com experiências passadas.\n\nTítulo: Sensores e Sistemas de Percepção\nDescrição: Radar, LiDAR, câmeras e sensores ultrassônicos que permitem aos carros autônomos \"ver\" e interpretar o ambiente.\n\nTítulo: \nDescrição:\n"



In [ ]:
#lista os modelos relacionados a geração de conteúdo
print("-------------------------------------------------------------------------")
print("---------------------- generate content models ---------------------------")
print("-------------------------------------------------------------------------")
for m in genai.list_models():
  if('generateContent') in m.supported_generation_methods:
    print(m.name)
print("-------------------------------------------------------------------------")
print("---------------------- embed content models -----------------------------")
print("-------------------------------------------------------------------------")
#lista os modelos para pesquisa de conteúdo
for m in genai.list_models():
  if('embedContent') in m.supported_generation_methods:
    print(m.name)

In [ ]:
# Inicialização da biblioteca generative ia
generative_model = "models/gemini-1.5-pro-latest"
model = genai.GenerativeModel(model_name=generative_model,
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

In [ ]:
# Inicialização do chat
chat = model.start_chat(history=[])

In [ ]:
# pesquisa conforme as palavras chave informadas
prompt = input("Informe as palavras chave, separadas por vírgula: ")
response = chat.send_message(prompt)

In [ ]:
# Carrega o resultado da pesquisa
data = json.loads(response.text)

# Cria o dataframe
df = pd.DataFrame(data)
df.columns = ["title", "content"]

# Exibe o dataframe
df

In [ ]:
#modelo de pesquisa de conteúdo
embedding_model = "models/embedding-001"

In [ ]:
#função que monta a coluna para pesquisa de conteúdo
def embedding_column_value(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
# Preenche o campo search no dataframe com os dados de pesquisa
df["search"] = df.apply(lambda row: embedding_column_value(row["title"], row["content"]), axis=1)
df

In [ ]:
# função utilizada para consulta de dados, de acordo com a pergunta feita
def search_data(search, base, model):
  result = genai.embed_content(model=model,
                               content=search,
                               task_type="RETRIEVAL_QUERY")["embedding"]

  scalar_result = np.dot(np.stack(df["search"]), result)

  index = np.argmax(scalar_result)
  if(index > -1):
    return df.iloc[index]["content"]
  else:
    return f"Conteúdo não localizado contendo a pesquisa {search}"

In [ ]:
search = input("Informe os dados da pesquisa: ")

result = search_data(search, df, embedding_model)
print(result)